#  RAG 성능 평가

---


## 1. 개요 및 환경 설정

### 1.1 RAG 평가가 중요한 이유

- **RAG(Retrieval-Augmented Generation)** 시스템은 외부 지식을 검색하여 LLM의 응답 품질을 향상시키는 기술
- RAG 시스템의 성능을 객관적으로 측정하고 개선하기 위해서는 체계적인 평가가 필요

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/rag_evaluation.png" alt="rag" align="center" border="0"  width="1000" height=auto>
</center>

[출처] https://arxiv.org/abs/2405.07437

### 1.2 환경 설정

- uv add seaborn

In [ ]:
# 환경변수 설정
from dotenv import load_dotenv
import os
load_dotenv()

# 기본 라이브러리
import json
import pandas as pd
import numpy as np
from pprint import pprint
from glob import glob

# LangSmith 추적 설정 (선택사항)
print(f"LangSmith 추적: {os.getenv('LANGSMITH_TRACING')}")

---

## 2. RAG 평가의 핵심 개념

- **평가 지표**

   | 평가 영역 | 세부 지표 | 설명 |
   |-----------|-----------|------|
   | **검색 (Retrieval)** | Context Relevancy | 검색된 문서가 질문과 얼마나 관련있는가? |
   |  | Context Recall | 정답에 필요한 모든 정보가 검색되었는가? |
   | **생성 (Generation)** | Faithfulness | 생성된 답변이 검색된 문서에 충실한가? |
   |  | Answer Relevancy | 생성된 답변이 질문과 관련있는가? |
   | **종합** | Answer Correctness | 생성된 답변이 정답과 일치하는가? |


- **평가 방법론**

   1. Reference-Free 평가
      - **장점**: 정답 데이터 없이도 평가 가능
      - **방법**: LLM-as-Judge 방식 활용
      - **도구**: RAGAS, LangSmith 등

   1. Reference-Based 평가
      - **장점**: 객관적이고 일관된 평가
      - **방법**: 정답과 비교하여 평가
      - **지표**: BLEU, ROUGE, Semantic Similarity 등

---

## 3. 평가 대상: 기본 RAG 시스템 구축

### 3.1 문서 준비 및 처리

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_documents(file_paths):
    """텍스트 파일들을 로드하는 함수"""
    documents = []
    for path in file_paths:
        try:
            loader = TextLoader(path, encoding='utf-8')
            documents.extend(loader.load())
        except Exception as e:
            print(f"파일 로드 실패 {path}: {e}")
    return documents

# 예시: 한국어 문서 로드
korean_files = glob('./data/*_KR.md')
documents = load_documents(korean_files)

print(f"로드된 문서 수: {len(documents)}")
if documents:
    print(f"첫 번째 문서 미리보기:\n{documents[0].page_content[:200]}...")

### 3.2 문서 분할 (Text Splitting)


In [ ]:
# 한국어 텍스트에 최적화된 분할기 설정
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],  # 문장 단위 분할
    chunk_size=300,           # 청크 크기
    chunk_overlap=0,         # 중복 영역
    is_separator_regex=True,  # 정규식 사용
    keep_separator=True       # 구분자 유지
)

# 문서 분할 실행
split_docs = text_splitter.split_documents(documents)

print(f"분할된 청크 수: {len(split_docs)}")
print(f"\n첫 번째 청크:")
print(f"메타데이터: {split_docs[0].metadata}")
print(f"내용: {split_docs[0].page_content}")

### 3.3 벡터 스토어 생성

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536  # 선택사항: 차원 축소로 성능 향상
)

# Chroma 벡터 스토어 생성
vector_store = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="rag_evaluation_demo",
    persist_directory="./local_chroma_db",
    collection_metadata={'hnsw:space': 'cosine'}  # 코사인 유사도 사용
)

print(f"벡터 스토어에 저장된 문서 수: {vector_store._collection.count()}")

### 3.4 RAG 체인 구성


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4.1-mini",  # 비용 효율적인 모델
    temperature=0,        # 일관된 결과를 위해 0으로 설정
)

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536  # 선택사항: 차원 축소로 성능 향상
)

# Chroma 벡터 스토어 로드
vector_store = Chroma(
    collection_name="rag_evaluation_demo",
    persist_directory="./local_chroma_db",
    embedding_function=embedding_model
)

print(f"벡터 스토어에 저장된 문서 수: {vector_store._collection.count()}")

# 검색기 설정
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 5개 문서 검색
)

# 프롬프트 템플릿
prompt_template = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답하세요. 
컨텍스트에 없는 정보는 추측하지 마세요.

<context>
{context}
</context>

<question>
{question}
</question>
""")

# RAG 체인 구성
def format_docs(docs):
    """문서 리스트를 문자열로 변환"""
    return "\n\n".join(doc.page_content for doc in docs)

def rag_chain(question: str) -> dict:
    """RAG 체인 실행 함수"""

    # 문서 검색
    retrieved_docs = retriever.invoke(question)
    
    # 컨텍스트 준비
    context = format_docs(retrieved_docs)
    
    # LLM으로 답변 생성
    response = llm.invoke(
        prompt_template.format_prompt(
            context=context, 
            question=question
        )
    ).content
    
    return {
        "question": question,
        "context": context,
        "answer": response,
        "retrieved_docs": retrieved_docs
    }

# 테스트 실행
test_question = "리비안의 설립자는 누구인가요?"
result = rag_chain(test_question)

print(f"질문: {result['question']}")
print(f"답변: {result['answer']}")
print(f"검색된 문서 수: {len(result['retrieved_docs'])}")

---

## 4. 평가 데이터셋 합성

### 4.1 RAGAS 사용 데이터 합성

- uv add ragas==0.3.1 rapidfuzz

- RAGAS 프레임워크 소개

    - **RAGAS (Retrieval-Augmented Generation Assessment)** 는 RAG 시스템을 위한 오픈소스 평가 프레임워크
    - **주요 특징**
        - ✅ **Reference-Free**: 정답 없이도 평가 가능
        - ✅ **LLM-as-Judge**: GPT-4 등을 활용한 자동 평가
        - ✅ **구성요소별 평가**: 검색과 생성을 개별적으로 평가
        - ✅ **LangChain 통합**: 기존 RAG 파이프라인과 쉽게 연동

    - 주요 지표
        - **Context Relevancy**: 검색된 문서가 질문과 얼마나 관련있는지 측정
        - **Context Recall**: 정답 생성에 필요한 정보가 얼마나 검색되었는지 측정
        - **Faithfulness**: 생성된 답변이 검색된 문서에 얼마나 충실한지 측정
        - **Answer Relevancy**: 생성된 답변이 질문과 얼마나 관련있는지 측정
        - **Answer Correctness**: 생성된 답변이 정답과 얼마나 일치하는지 측정
        

In [ ]:
from ragas.testset import TestsetGenerator
from ragas.testset.persona import Persona
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# LLM과 임베딩 래퍼 설정
generator_llm = LangchainLLMWrapper(
    ChatOpenAI(model="gpt-4.1-mini", temperature=0.2)
)
generator_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(model="text-embedding-3-small")
)

# 한국어 페르소나 정의 
personas = [
    Persona(
        name="graduate_researcher",
        role_description="""미국 전기차 시장을 연구하는 한국인 박사과정 연구원입니다. 
        전기차 정책, 시장 동향, 기술적 세부사항에 대해 깊이 있는 분석적 질문을 합니다. 
        학술적 용어를 사용하며 데이터와 근거를 중요하게 생각합니다. 한국어만 사용합니다.""",
    ),
    Persona(
        name="masters_student",
        role_description="""전기차 산업을 공부하는 한국인 석사과정 학생입니다. 
        미국 전기차 시장의 기초 개념과 트렌드를 이해하려 노력하며, 
        명확하고 이해하기 쉬운 설명을 선호합니다. 한국어만 사용합니다.""",
    ),
    Persona(
        name="industry_analyst",
        role_description="""한국 자동차 회사에서 미국 전기차 시장을 분석하는 주니어 연구원입니다. 
        실무적인 시장 데이터, 경쟁사 동향, 비즈니스 인사이트에 관심이 많으며, 
        실행 가능한 정보를 중요하게 생각합니다. 한국어만 사용합니다.""",
    ),
    Persona(
        name="policy_researcher",
        role_description="""한국 정부기관에서 전기차 정책을 연구하는 연구원입니다. 
        미국의 전기차 관련 정책, 인센티브, 규제에 대해 관심이 많으며, 
        한국 정책에 적용 가능한 시사점을 찾고 있습니다. 한국어만 사용합니다.""",
    )
]


In [ ]:
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer

# 기본 프롬프트 (영어 버전) 확인
synthesizer = SingleHopSpecificQuerySynthesizer(llm=generator_llm)

synthesizer.get_prompts()

In [ ]:
# 한국어 프롬프트로 변환
korean_prompts = await synthesizer.adapt_prompts(
    language="korean", 
    llm=generator_llm
)
synthesizer.set_prompts(**korean_prompts)

In [ ]:
print(korean_prompts['query_answer_generation_prompt'].instruction)

In [ ]:
synthesizer.get_prompts()

In [ ]:
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer
from ragas.testset.synthesizers.multi_hop.specific import MultiHopSpecificQuerySynthesizer
from ragas.testset.synthesizers.multi_hop.abstract import MultiHopAbstractQuerySynthesizer


async def create_korean_query_distribution():
    """한국어 최적화된 Query Distribution 생성"""
    
    # Query Synthesizer들을 한국어로 적응
    synthesizers = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.6),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.2),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.2)
    ]
    
    # 각 synthesizer의 프롬프트를 한국어로 적응
    korean_query_distribution = []
    for synthesizer, weight in synthesizers:

        # 한국어 프롬프트 적응
        korean_prompts = await synthesizer.adapt_prompts(
            language="korean", 
            llm=generator_llm
        )
        synthesizer.set_prompts(**korean_prompts)
        print(f"{synthesizer.__class__.__name__} 한국어 적응 완료")
        korean_query_distribution.append((synthesizer, weight))

    return korean_query_distribution

In [ ]:
# 합성 데이터셋 생성
async def generate_korean_testset(split_docs, testset_size=60):
    """한국어 테스트셋 생성"""
    
    # 1. 한국어 적응된 Query Distribution 생성
    korean_query_distribution = await create_korean_query_distribution()
    
    # 2. TestsetGenerator 생성
    testset_generator = TestsetGenerator(
        llm=generator_llm,
        embedding_model=generator_embeddings,
        persona_list=personas
    )
    
    # 3. 테스트셋 생성 (query_distribution 매개변수 사용)
    synthetic_dataset = testset_generator.generate_with_langchain_docs(
        documents=split_docs,
        testset_size=testset_size,
        query_distribution=korean_query_distribution  # 올바른 매개변수
    )
    
    return synthetic_dataset


synthetic_dataset = await generate_korean_testset(split_docs, testset_size=50)

In [ ]:
# 결과 확인
df = synthetic_dataset.to_pandas()
print(f"생성된 합성 데이터셋 크기: {len(df)}")
print(f"\n컬럼: {list(df.columns)}")
print(f"\n첫 번째 샘플:")
print(f"질문: {df.iloc[0]['user_input']}")
print(f"정답: {df.iloc[0]['reference']}")

# CSV로 저장
df.to_csv('./data/synthetic_testset.csv', index=False, encoding='utf-8')

In [ ]:
df.head()

### 4.2 평가용 데이터셋 준비

In [ ]:
from ragas import EvaluationDataset

def create_evaluation_dataset(questions_data, rag_chain_func):
    """평가용 데이터셋을 생성하는 함수"""
    
    evaluation_data = []
    
    # 각 질문에 대해 RAG 체인을 실행하고 평가 데이터 구성
    for item in questions_data:

        question = item["user_input"]
        ground_truth = item["reference"]

        # RAG 체인 실행
        result = rag_chain_func(question)
        
        # 평가 데이터 구성
        eval_sample = {
            "user_input": question,
            "response": result["answer"],
            "retrieved_contexts": [doc.page_content for doc in result["retrieved_docs"]],
            "reference": ground_truth
        }
        
        evaluation_data.append(eval_sample)
    
    return EvaluationDataset.from_list(evaluation_data)


# 평가 데이터셋 준비
evaluation_questions = synthetic_dataset.to_list()  # 합성 데이터셋을 평가 질문으로 사용

print(f"총 {len(evaluation_questions)}개의 평가 질문이 있습니다.")

In [ ]:
# 평가 데이터셋 생성
eval_dataset = create_evaluation_dataset(evaluation_questions, rag_chain) 

# 데이터프레임으로 확인
eval_df = eval_dataset.to_pandas()
print(f"\n평가 데이터셋 생성 완료:")
print(f"- 샘플 수: {len(eval_df)}")
print(f"- 컬럼: {list(eval_df.columns)}")

# 저장
eval_df.to_csv('./data/evaluation_dataset.csv', index=False, encoding='utf-8')

In [ ]:
eval_df

---

## 5. RAGAS 평가 수행

- **RAG 평가 지표 (예시)**

    ```python
    from ragas.metrics import (
        context_relevancy,      # 컨텍스트 관련성
        context_recall,         # 컨텍스트 회상률
        faithfulness,          # 충실도
        answer_relevancy,      # 답변 관련성
        answer_correctness     # 답변 정확성
    )
    ```

- **지표별 상세 설명**

    1. **Context Relevancy (컨텍스트 관련성)**
        - 검색된 문서가 질문과 얼마나 관련있는지 측정
        - 계산식: `관련 문장 수 / 전체 문장 수`

    2. **Context Recall (컨텍스트 검출률)**
        - 정답 생성에 필요한 정보가 얼마나 검색되었는지 측정
        - 정답(ground truth) 필요

    3. **Faithfulness (충실도)**
        - 생성된 답변이 검색된 문서에 얼마나 충실한지 측정
        - 환각(hallucination) 검출에 중요

    4. **Answer Relevancy (답변 관련성)**
        - 생성된 답변이 질문과 얼마나 관련있는지 측정

    5. **Answer Correctness (답변 정확성)**
        - 생성된 답변이 정답과 얼마나 일치하는지 측정

### 5.1 기본 평가 실행

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    LLMContextRecall,
    Faithfulness, 
    AnswerRelevancy,
    ContextPrecision,
    FactualCorrectness
)

# 평가용 LLM 설정
evaluator_llm = LangchainLLMWrapper(
    ChatOpenAI(model="gpt-4.1-mini", temperature=0)
)

# 평가 지표 선택
metrics = [
    LLMContextRecall(llm=evaluator_llm),           # 컨텍스트 검출율
    Faithfulness(llm=evaluator_llm),               # 충실도
    AnswerRelevancy(llm=evaluator_llm),            # 답변 관련성
    ContextPrecision(llm=evaluator_llm),           # 컨텍스트 정밀도
    FactualCorrectness(llm=evaluator_llm)          # 사실적 정확성
]

print("RAGAS 평가 시작...")

# 평가 실행
results = evaluate(
    dataset=eval_dataset[:10],  # 처음 10개 샘플로 평가 (테스트 목적)
    metrics=metrics,
    llm=evaluator_llm,
    embeddings=generator_embeddings
)

print("평가 완료!")
print(f"\n전체 평가 결과:")
print(results)

### 5.2 상세 결과 분석


In [ ]:
# 결과를 DataFrame으로 변환
results_df = results.to_pandas()

print(f"\n상세 평가 결과:")
print("="*80)

# 각 샘플별 상세 결과
for idx, row in results_df.iterrows():
    print(f"\n[샘플 {idx+1}]")
    print(f"질문: {row['user_input'][:100]}...")
    print(f"답변: {row['response'][:100]}...")
    
    # 지표별 점수 출력
    for col in results_df.columns:
        if col not in ['user_input', 'response', 'retrieved_contexts', 'reference']:
            if pd.notna(row[col]):
                print(f"  {col}: {row[col]:.3f}")
    print("-" * 50)

In [ ]:
# 통계 요약
print(f"\n📊 평가 지표 통계:")
print("="*50)

numeric_columns = results_df.select_dtypes(include=[np.number]).columns 
summary_stats = results_df[numeric_columns].describe()

for metric in numeric_columns:
    mean_score = summary_stats.loc['mean', metric]
    std_score = summary_stats.loc['std', metric]
    print(f"{metric}:")
    print(f"  평균: {mean_score:.3f} (±{std_score:.3f})")
    print(f"  범위: {summary_stats.loc['min', metric]:.3f} ~ {summary_stats.loc['max', metric]:.3f}")


## [참고] 메트릭별 핵심 평가 요소

| 메트릭 | 평가 대상 | 핵심 질문 | 정의 |
|--------|-----------|-----------|------|
| **LLMContextRecall** | 검색 품질 | "필요한 정보가 잘 검색되었는가?" | 참조 답변(reference)의 주장들이 검색된 컨텍스트에 의해 얼마나 잘 지원되는지를 측정하는 지표 |
| **Faithfulness** | 생성 품질 | "답변이 컨텍스트에 충실한가?" | 생성된 답변이 검색된 컨텍스트에 얼마나 충실한지(일치하는지) 측정하는 지표 |
| **AnswerRelevancy** | 답변 품질 | "답변이 질문과 관련이 있는가?" | 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지 측정하는 지표 |
| **ContextPrecision** | 검색 정밀도 | "검색된 것들이 실제로 유용한가?" | 검색된 컨텍스트 중 질문과 실제로 관련된 것들의 비율을 측정하는 지표 |
| **FactualCorrectness** | 사실 정확성 | "답변이 사실적으로 정확한가?" | 생성된 응답과 참조 답변 간의 사실적 정확성을 측정하는 지표 |

In [ ]:
# 결과 저장
results_df.to_csv('./data/ragas_evaluation_results.csv', index=False, encoding='utf-8')